# ワークショップ前の手順

*ワークショップの前に* 次の手順を実行してください。

## ステップ 1: デスクトップ環境に最新のCytoscape (3.9.0) をインストールします
- [ここから Cytoscape のインストーラをダウンロード](https://cytoscape.org/download.html)。
- Javaが無ければ合わせてインストールしてくれます。
- スタートメニューまたはデスクトップショートカットからCytoscape3.9.0を起動します

![image](https://user-images.githubusercontent.com/12192/139441980-5d73579f-39dd-45da-916b-759eff99474d.png)

- 上の画像のようなメッセージが表示されますので、OKボタンを押してCytoscapeを再起動してください。
- Cytoscape は立ち上げっぱなしにしておいてください。

## ステップ 2: Docker 環境の立ち上げと Google Chrome を用いたその環境へのアクセス
- `docker run -d -p 8888:8888 kozo2/cyautoworkshop` をあなたのPCで実行します。
- Google Chrome で `http://localhost:8888` を開きます。Safariでは動作しません。Firefoxは動作するかもしれませんが確認していません。

# ワークショップ (入門編)

## 自己紹介

西田 孝三

- Bioconductor Community Advisory Board (CAB) のメンバー
- RCy3 を用いた Bioconductor package (transomics2cytoscape) の作者
- Cytoscape のコミュニティへの貢献 (Google Summer of Code, Google Season of Docs)
- KEGGscape Cytoscape App の KEGGscape の作者

## Cytoscape とは何ですか?

![image](https://user-images.githubusercontent.com/12192/139426468-915e9a76-7e4e-4a37-aee9-3d0e344f551e.png)

- オープンソースの、クロスプラットフォームの、JavaデスクトップGUIアプリ。
- ネットワーク可視化のためのもの。

### コアコンセプト

**ネットワークとテーブル**: ネットワークのノードとエッジはアノテーションのテーブルを持っています。

![image](https://user-images.githubusercontent.com/12192/139427094-bfd9a839-dabf-468d-8f28-6458443c8e61.png)

![image](https://user-images.githubusercontent.com/12192/139427149-4f0fe568-3851-4de6-834e-2e809e85f1be.png)

色、形、サイズ、他何らか、といった「見栄え」をアノテーションの値によってスタイリングします。

## なぜ自動化する必要があるんですか?
GUIを直接使用できるのに、なぜCytoscapeを自動化するのですか？

- ループなど、複数回実行したいことのために
- 将来繰り返したいこと(再現)のために
- 同僚と共有したり公開したりしたいもののために
- RやPythonなどですでに取り組んでいるものを活かすために

端的には「再現性」「データ共有」「RかPythonの資源活用」のため。

## Cytoscape GUI操作はどのように自動化を実現しているのですか?

![image](https://user-images.githubusercontent.com/12192/139397677-80076550-e458-4bd4-9ab5-ba48ef6843b9.png)

- REST APIでそれを可能にしています。ほとんどすべてのGUI 操作がREST APIで行えるようになっています。
- 昨今のCytoscapeはデスクトップアプリケーションであるだけでなく、RESTサーバーでもあります。
- 以下のコマンドで、Cytoscapeがサーバーとして機能しているかどうかを確認できます。

  ```
  curl localhost:1234
  ```

- RCy3 または py4cytoscape といったパッケージはそのREST APIのラッパーです。
- py4cytoscape は RCy3 の Pythonクローンです。最近できました。py4cytoscape は RCy3 と同じ関数仕様を持っているため、片方使えると両方使えるようになったのと同じです。
- バイオインフォマティクスにはテーブル操作が不可欠であるため、RやPythonでCytoscapeを操作できると便利です。dplyrやpandasがあるからです。そういったテーブル操作をCytoscapeのGUIで行うのは面倒です。

[CyREST: Turbocharging Cytoscape Access for External Tools via a RESTful API. F1000Research 2015.](https://dx.doi.org/10.12688%2Ff1000research.6767.1)

[Cytoscape Automation: empowering workflow-based network analysis. Genome Biology 2019.](https://doi.org/10.1186/s13059-019-1758-4)

## RCy3を用いた自動化

![image](https://user-images.githubusercontent.com/12192/139400142-8a2a764b-dbbe-4e47-9d3c-d4cc07602468.png)

[RCy3: Network biology using Cytoscape from within R. F1000Research 2019.](https://f1000research.com/articles/8-1774)

## RCy3を使用してRのデータをCytoscapeネットワークに変換する

ネットワークは、生物学的データを表すための便利な方法を提供してくれます。
しかし、データをRからCytoscapeにシームレスに変換するにはどうすればよいでしょうか?

![image](https://user-images.githubusercontent.com/12192/139404069-536a67a2-e8fe-4072-bc42-74bfb060f924.png)

ここからいよいよハンズオンです。細かいことはおいておいてとにかくRとCytoscapeを接続しましょう。

以下のコードを実行する前に、ローカルのCytoscapeが完全に起動しきっていることを確認してください。
Cytoscapeが起動し、RESTサーバーが完全に起動するまでには少し時間がかかります。
(大体10秒くらい待ちます。)

In [ ]:
library(RCy3)
browserClientJs <- getBrowserClientJs()
IRdisplay::display_javascript(browserClientJs)
cytoscapePing()

### なぜ Jupyter が Cytoscape の REST サービスと通信できたのですか？

下記で何が起こったのかについての詳細な説明が必要ですね。

```
browserClientJs <- getBrowserClientJs()
IRdisplay::display_javascript(browserClientJs)
```

上記のコードでは、**Jupyter Bridge** と呼ばれるテクノロジーを使用しました。
Jupyter Bridgeは、リモートの REST クライアントからの HTTP リクエストをローカルリクエストのように見せかける JavaScript 実装です。

![image](https://user-images.githubusercontent.com/12192/139530994-8afd99b2-1175-46b3-9ad7-166d8ba78f2a.png)

デスクトップ環境内のCytoscapeにリモート環境からアクセスするのは難しいため、Jupyter Bridgeを使用します。

そしてこのワークショップが他のワークショップと違って [Orchestra](http://app.orchestra.cancerdatasci.org/) で動かなかったのは
この Jupyter Bridge のせいでもあります。

そのため私はローカルの Docker 環境内の Jupyter で Jupyter Bridge を使いました。
ローカルでなくても Orchestra でないリモートサーバーで私の Docker イメージを使うなら問題無く動作します。

### (で) なぜ Jupyter Bridge を使うのですか? ややこしくないですか? ノートブックを使わなければよいのではないですか?

- ユーザーは依存関係や環境について心配する必要が無いからです。
- ノートブックベースのワークフローとデータセットを簡単に共有できるからです。
- ワークフローはクラウドに常駐し、クラウドリソースにアクセスし、それでもCytoscape機能を使用できるところがよいのです。

### RデータをCytoscapeネットワークに変換する方法に戻りましょう...

いくつかの基本的なRオブジェクトからCytoscapeネットワークを作成します


In [ ]:
nodes <- data.frame(id=c("node 0","node 1","node 2","node 3"),
    group=c("A","A","B","B"), # categorical strings
    score=as.integer(c(20,10,15,5)), # integers
    stringsAsFactors=FALSE)

In [ ]:
nodes

In [ ]:
edges <- data.frame(source=c("node 0","node 0","node 0","node 2"),
    target=c("node 1","node 2","node 3","node 3"),
    interaction=c("inhibits","interacts","activates","interacts"),  # optional
    weight=c(5.1,3.0,5.2,9.9), # numeric
    stringsAsFactors=FALSE)

In [ ]:
edges

#### ネットワークの作成に使用されるデータフレーム

![image](https://user-images.githubusercontent.com/12192/139534280-0c569dfd-d66d-4054-9b58-becce79225bc.png)

#### さあ、RCy3からネットワークを作りましょう

In [ ]:
createNetworkFromDataFrames(nodes, edges, title="my first network", collection="DataFrame Example")

#### ネットワークの画像をエクスポートしましょう

大部分の人の目的はCytoscapeの画を論文のFigureに使うことでしょう。
ですので画像を取得する方法が知りたいでしょう。

In [ ]:
exportImage("my_first_network", type = "png")

In [ ]:
library(RCurl)
matrix <- getURL("https://raw.githubusercontent.com/cytoscape/cytoscape-tutorials/gh-pages/presentations/modules/RCy3_ExampleData/data/TCGA_OV_RNAseq_All_edgeR_scores.txt")
RNASeq_gene_scores <- read.table(text=matrix, header = TRUE, sep = "\t", quote="\"", stringsAsFactors = FALSE)

In [ ]:
RNASeq_gene_scores